In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext sql

In [2]:
%sql sqlite:///results.db

# Code for Generating the Big Table of Commands Broken Down by Customization Practice

Some stuff is spread out over the other notebooks.

In [3]:
%%sql
create view command_alias_totals as
select cmd_name, count(distinct alias_id) as total
    from (
        select command.name as cmd_name, alias_id from command
        union
        select alias.name as cmd_name, alias_id from alias where alias.name in known_commands
    )
    group by cmd_name

 * sqlite:///results.db
Done.


[]

In [7]:
%%sql
drop table if exists selected_commands;
create temporary table selected_commands (command_synonym_id integer, cmd_name text);
insert into selected_commands values
(1,'git'),
(1,'/usr/bin/git'),
(2,'hg'),
(3,'grep'),
(3,'egrep'),
(3,'fgrep'),
(4,'sort'),
(5,'vi'),
(5,'vim'),
(5,'nvim'),
(5,'mvim'),
(6,'emacs'),
(6,'emacsclient'),
(7,'sed'),
(8,'subl'),
(9,'nano'),
(10,'ls'),
(11,'cd'),
(12,'rm'),
(13,'cp'),
(14,'mv'),
(15,'du'),
(16,'mkdir'),
(17,'df'),
(18,'docker'),
(18,'docker-compose'),
(18,'docker-machine'),
(19,'kubectl'),
(19,'k'),
(20,'vagrant'),
(21,'ssh'),
(22,'curl'),
(23,'wget'),
(24,'apt'),
(24,'apt-get'),
(24,'apt-cache'),
(24,'aptitude'),
(24,'$apt_pref'),
(25,'pacman'),
(26,'brew'),
(27,'tmux');

 * sqlite:///results.db
Done.
Done.
41 rows affected.


[]

In [9]:
%%sql bigtable <<

with cmds as (select cmd_name from selected_commands)

select 

'',
'\texttt{{' || cmd_name || '}}' as 'Command',
'\num{{' || sum(total) || '}}' as '# Aliases',
'',
'\pie{{' || round(sum(nicknaming)*100.0/sum(total),2) || '}}' as 'Nicknaming Commands',
'\pie{{' || round(sum(abbreviating)*100.0/sum(total),2) || '}}' as 'Abbreviating Subcommands',
'\pie{{' || round(sum(bookmarking)*100.0/sum(total),2) || '}}' as 'Bookmarking Locations',
'',
'\pie{{' || round(sum(substituting)*100.0/sum(total),2) || '}}' as 'Substituting Commands',
'\pie{{' || round(sum(overriding)*100.0/sum(total),2) || '}}' as 'Overriding Defaults',
'\pie{{' || round(sum(colorizing)*100.0/sum(total),2) || '}}' as 'Colorizing Output',
'\pie{{' || round(sum(elevating)*100.0/sum(total),2) || '}}' as 'Elevating Privilege',
'',
'\pie{{' || round(sum(transforming)*100.0/sum(total),2) || '}}' as 'Transforming Data',
'\pie{{' || round(sum(chaining)*100.0/sum(total),2) || '}}' as 'Chaining Subcommands',
'',
'\hist{{' || cmd_name || '}}' as 'Compression Ratio'

from

(select * from command_alias_totals where cmd_name in cmds)

left outer join (select * from selected_commands) using (cmd_name)

left outer join (

select command.name as cmd_name, count(distinct alias_id) as nicknaming
from alias join command using (alias_id)
where alias.value = command.name
and alias.name != command.name
and alias.name not in known_commands
and cmd_name in cmds group by cmd_name

) using (cmd_name) left outer join (    

    select command.name as cmd_name, count(distinct alias_id) as abbreviating
    from alias join command using (alias_id)
    where alias.num_commands = 1
    and command.num_arguments = 1
    and command.name in commands_with_subcommands
    and cmd_name in cmds group by cmd_name
    
) using (cmd_name) left outer join (

    select command.name as cmd_name, count(distinct alias_id) as bookmarking
    from argument
    join bookmark using (argument_id)
    join command using (command_id)
    where cmd_name in cmds group by cmd_name

) using (cmd_name) left outer join (

    select cmd_name, count(distinct alias_id) as substituting from (

        select alias.name as cmd_name, alias_id
        from alias
        where alias.name in known_commands
        and alias.value != alias.name
        and alias.value not like alias.name || ' %'
        and alias.value not like '% ' || alias.name
        and alias.value not like '% ' || alias.name || ' %'
        and cmd_name in cmds
    
        union
    
        select command.name as cmd_name, alias_id
        from alias join command using (alias_id)
        where alias.name in known_commands
        and alias.value != alias.name
        and alias.value not like alias.name || ' %'
        and alias.value not like '% ' || alias.name
        and alias.value not like '% ' || alias.name || ' %'
        and cmd_name in cmds

    ) group by cmd_name

) using (cmd_name) left outer join (

    select command.name as cmd_name, count(distinct alias_id) as overriding
    from alias join command using (alias_id)
    where alias.name = command.name
    and cmd_name in cmds group by cmd_name

) using (cmd_name) left outer join (

    select cmd_name, count(distinct alias_id) as colorizing from (
    
        select command.name as cmd_name, alias_id
        from alias 
        join aliases_colorizing_output using (alias_id) 
        join command using (alias_id)
        where command.name not like '%=%'
        and cmd_name in cmds
    
        union
    
        select alias.name as cmd_name, alias_id
        from alias
        join aliases_colorizing_output using (alias_id)
        where alias.name in known_commands
        and cmd_name in cmds
    
        union
    
        select argument.name as cmd_name, alias_id
        from alias
        join aliases_colorizing_output using (alias_id) 
        join command using (alias_id)
        join argument using (command_id)
        where argument.name in known_commands
        and cmd_name in cmds

    ) group by cmd_name

) using (cmd_name) left outer join (

    select command.name as cmd_name, count(distinct alias_id) as elevating
    from command
    where command.sudo = 1
    and command.name not like '-%'
    and cmd_name in cmds group by cmd_name

) using (cmd_name) left outer join (

    select command.name as cmd_name, count(distinct alias_id) as transforming
    from command
    where command.operator = '|'
    and cmd_name in cmds group by cmd_name

) using (cmd_name) left outer join (

    select c1.name as cmd_name, count(distinct alias_id) as chaining
    from alias 
    join command c1 using (alias_id)
    join command c2 using (alias_id)
    where c1.name in commands_with_subcommands
    and c1.name = c2.name
    and c2.position = c1.position + 1
    and (c2.operator = '&&' or c2.operator = ';')
    and cmd_name in cmds group by cmd_name

) using (cmd_name)


group by command_synonym_id

 * sqlite:///results.db
Done.
Returning data to local variable bigtable


In [10]:
print(bigtable.DataFrame().to_latex(index=False, escape=False, column_format='llrlllllllllllllccc'))

\begin{tabular}{llrlllllllllllllccc}
\toprule
'' &                Command &     # Aliases & '' & Nicknaming Commands & Abbreviating Subcommands & Bookmarking Locations & '' & Substituting Commands & Overriding Defaults & Colorizing Output & Elevating Privilege & '' & Transforming Data & Chaining Subcommands & '' &    Compression Ratio \\
\midrule
   &  \texttt{/usr/bin/git} &  \num{315841} &    &          \pie{3.44} &              \pie{36.11} &            \pie{1.84} &    &            \pie{0.97} &          \pie{0.04} &        \pie{1.26} &          \pie{0.03} &    &        \pie{0.05} &           \pie{3.82} &    &  \hist{/usr/bin/git} \\
   &            \texttt{hg} &    \num{2799} &    &          \pie{2.47} &              \pie{44.52} &            \pie{2.22} &    &           \pie{22.15} &          \pie{0.57} &        \pie{1.68} &                None &    &        \pie{0.07} &           \pie{3.43} &    &            \hist{hg} \\
   &         \texttt{egrep} &  \num{144606} &    &          \pi